<a href="https://colab.research.google.com/github/bhoaw/ML-Supervised-Prediction-of-Driving-Conditions/blob/main/metodi_prova_INT8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def quantize_tensor(x, num_bits=8, symmetric=True):
    """
    Quantizza un tensore float32 in int8.
    Per la quantizzazione simmetrica, l'intervallo di int8 è [-128, 127]
    """
    if symmetric:
        qmin, qmax = -128, 127
        max_val = max(abs(x.min()), abs(x.max()))
        scale = max_val / float(qmax) if max_val != 0 else 1.0
        zero_point = 0
    else:
        qmin, qmax = 0, 255
        rmin, rmax = x.min(), x.max()
        scale = (rmax - rmin) / (qmax - qmin) if (rmax - rmin) != 0 else 1.0
        zero_point = int(np.round(qmin - rmin / scale))
    q_x = np.round(x / scale).astype(np.int32) + zero_point
    q_x = np.clip(q_x, qmin, qmax).astype(np.int8)
    return q_x, scale, zero_point

def dequantize_tensor(q_x, scale, zero_point):
    """ Riporta un tensore quantizzato al formato float32 """
    return scale * (q_x.astype(np.int32) - zero_point)

In [ ]:
def relu(x, leak = 0):                      # ReLU
    return np.where(x <= 0, leak * x, x)

def relu_dash(x, leak = 0):                 # ReLU derivative
    return np.where(x <= 0, leak, 1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def d_sigmoid(x):
    #return (np.exp(-x))/((np.exp(-x)+1)**2)
    return (1 - sigmoid(x)) * sigmoid(x)

def softmax(x):
  #print("x.shape ", x.shape)
  # Numerically stable with large exponentials
  exps = np.exp(x - np.max(x, axis=0, keepdims=True)) #x.max())
  return exps / np.sum(exps, axis=0)

def d_softmax(x):
  exps = np.exp(x - np.max(x, axis=0, keepdims=True)) #x.max())
  return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))


def categorical_crossentropy(output, target, from_logits=False):
    # Log loss of the correct class of each of our samples
  #correct_logprobs = -np.log(probs[np.arange(n_samples), y])
  #10x100
  # target: batchXnum_class
  # output: num_classXbatch
   # print("---- sum: output.shape ", output.shape, " target.shape ", target.shape)
   # print("for target.shape[0] ", target.shape[0])
   # print("for target.shape[1] ", target.shape[1])
    #print("target_test ", target.shape) # [40, 10]
    #print("output ", output.shape) # [10, 40]
    vett = []
    if from_logits:
        output = softmax(output)
    else:
      for i in range (0,target.shape[0]): #batch
        #output /= output.sum(axis=0, keepdims=True)
        #output = np.clip(output, 1e-7, 1 - 1e-7)
        for j in range (0,target.shape[1]):
          if target[i,j] == 1:
            vett.append(-np.log(output[j,i]))
            break
        #vett.append(np.sum(target * -np.log(output), axis=-1, keepdims=False))
    return vett

In [ ]:
class Layer:
    activationFns = {
           "sigmoid": (sigmoid, d_sigmoid),
           "softmax": (softmax, d_softmax),
           "relu": (relu, relu_dash)
        }

    def __init__(self, inputs, neurons, activation, W_saved, b_saved, retrain): #
      if retrain == 0:
        np.random.seed(seed=42)
        W = (np.random.randn(neurons, inputs)*np.sqrt(1/neurons)).astype(np.float32)
        b = np.zeros((neurons, 1))
        self.act, self.d_act= self.activationFns.get(activation)
      if retrain == 1:
        W = W_saved
        b = b_saved
        self.act, self.d_act= self.activationFns.get(activation)

      self.qW, self.scaleW, _ = quantize_tensor(W, symmetric=True)
      self.qb, self.scaleb, _ = quantize_tensor(b, symmetric=True)



    def update(self, W, b, layer):
      if layer != 0:
        self.qW = W
      self.qb = b

    def feedforward(self, A1, layer):
      print("------------------- feedforward")
      self.A1 = A1
      qA1, scale_A1, _ = quantize_tensor(self.A1, symmetric=True); print("self.A1.shape ", self.A1.shape)

      Z = np.dot(self.qW.astype(np.int32), qA1.astype(np.int32)); #print("type(Z) ", type(self.Z))
      #z1 = np.dot(qx.astype(np.int32), self.qW1.astype(np.int32)) + self.qb1.astype(np.int32)
      Z = Z + np.array(self.qb).astype(np.int32) #.reshape(10, 1)
      self.Z = Z * (scale_A1 * self.scaleW); print("Z.shape ", self.Z.shape);
      self.A = self.act(self.Z); print("self.A.shape ", self.A.shape)
      return self.A

    def backprop(self, dA, learning_rate, layer, target, lambd, num_lyr):
      print()
      print()
      print("------------------- backpropagation")
      if layer == num_lyr:
        print("last layer")
        '''
        error = 2 * (output - y_train) / output.shape[0] * self.softmax(params['Z3'], derivative=True)
        change_w['W3'] = np.outer(error, params['A2'])
        '''
        #dZ = (dA-target.T);
        dZ_int32 = dA.astype(np.int32) - (target.T).astype(np.int32);
        print("dA.shape ", dA.shape); print("dZ.shape ", dZ_int32.shape); print("target.T[NUM_CLASS][BATCH_SIZE]: ", target.T.shape)
        dZ = np.clip(dZ_int32, -128, 127).astype(np.int8)
      else:
        print("layer 1 or 0")
        print("dA.shape ", dA.shape)
        #dZ = np.multiply(self.d_act(self.Z), dA) #relu #sigmoid
        d_act_float = self.d_act(self.Z);
        d_act_q, scale_dact, zp_dact = quantize_tensor(d_act_float, symmetric=True)
        dZ_int32 = d_act_q.astype(np.int32) * dA.astype(np.int32)
        dZ = np.clip(dZ_int32, -128, 127).astype(np.int8)

      #dW = (1.0/dZ.shape[1]) * np.matmul(dZ, np.transpose(self.A1));
      #dW += (lambd/dZ.shape[1])*self.W
      dW_int32 = (1.0/dZ.shape[1]) * np.matmul(dZ.astype(np.int32), (self.A1.T).astype(np.int32))
      dW_int32 += (lambd/dZ.shape[1]) * self.qW.astype(np.int32)
      dW_float = dW_int32.astype(np.float32)
      dW_q, scale_dW, zp_dW = quantize_tensor(dW_float, symmetric=True)
      print("dW.shape ", dW_q.shape); print("np.transpose(self.A1).shape ", np.transpose(self.A1).shape)

      #db = (1.0/dZ.shape[1]) * np.sum(dZ, axis=1, keepdims=True)
      db_int32 = (1.0/dZ.shape[1]) * np.sum(dZ.astype(np.int32), axis=1, keepdims=True)
      db_float = db_int32.astype(np.float32)
      db_q, scale_db, zp_db = quantize_tensor(db_float, symmetric=True)

      #dA1 = np.dot(self.W.T, dZ); print("self.W.T ", self.W.T.shape); print("dZ ", dZ.shape)
      dA1_int32 = np.dot(self.qW.T.astype(np.int32), dZ.astype(np.int32))
      dA1_float = dA1_int32.astype(np.float32)
      dA1_q, scale_dA1, zp_dA1 = quantize_tensor(dA1_float, symmetric=True)

      #self.W = (self.W - learning_rate * dW)
      #self.b = (self.b - learning_rate * db)
      W_float = self.qW.astype(np.float32)
      b_float = self.qb.astype(np.float32)

      W_updated = W_float - learning_rate * dW_q.astype(np.float32)
      b_updated = b_float - learning_rate * db_q.astype(np.float32)

      self.qW, self.scale_W, self.zp_W = quantize_tensor(W_updated, symmetric=True)
      self.qb, self.scale_b, self.zp_b = quantize_tensor(b_updated, symmetric=True)
      print("self.qW.shape ", self.qW.shape); print("type(qW) ", type(self.qW));

      return dA1_q, self.qW, self.qb



